# <center>**Pre-Processing**

Many of the requirements for this notebook have been fulfilled in the EDA. I will copy and paste over the code that I used in EDA for those parts. But first we must import packages and import the data.

## Import Packages

In [1]:
# To create dummy variables and manage dataset
import pandas as pd

# To scale the dataset.
from sklearn.preprocessing import StandardScaler

# To split the data into training vs. test set
from sklearn.model_selection import train_test_split

import os

In [2]:
# Find and set the working directory for this project
os.chdir(r'C:\Users\jdrel\OneDrive\Documents\Data_Science\Springboard\Capstone-2')

Load the Data

In [3]:
# Import the data
data = pd.read_csv('./data/interim/KDD Data.csv')
data = data.drop('Unnamed: 0', axis = 1)

Scale the data for Analysis

In [4]:
#Remove the y from the dataframe so the result is not influence by it
X = data.drop('labels', axis = 1)

# Create list of categorical columns including binary in a list comprehension. 
cat_col = [col for col in X.columns if X[col].dtype == 'object' or set(X[col].unique()) == {0,1}]

# Convert binary columns into categorical columns because they represent two states more so than degrees
X[cat_col] = X[cat_col].astype('category')

# Gather all of the columns in X that aren't in the categorical column
num_col = list(set(list(X.columns)) - set(cat_col))

# Create a dataframe that is just the numerical X columns
X_num = X[num_col]

# Use the sklearn's standard scalar and fit it to the data and return the scaled data.
X_num_scaled = pd.DataFrame(StandardScaler().fit_transform(X_num))
X_num_scaled.columns = X_num.columns

# Create X_scaled which is the scaled numerical columns and the categorical columns
X_scaled = pd.concat([X_num_scaled, X[cat_col]], axis= 1)

Transform Categorical variables into onehot features.

In [5]:
# import the data with K=7
dataK7 = pd.read_csv(r'./data/processed/data+k=7.csv')[['labels', 'k=7']]

#Create a dummy matrix for the types of intrusions and labels
lab_clus_dummy = pd.get_dummies(dataK7, columns = ['labels', 'k=7'], 
                                # No prefix to make the graph look better
                                prefix = ['', ''], 
                                # No sep to make the graph look better
                                prefix_sep = '')


Breakdown data into training vs. test set

In [10]:
# import data with k=7 full dataset
dataK7_full = pd.read_csv(r'./data/processed/data+k=7.csv')

#Split the data in to training set and a test set
X_train, X_test, y_train, y_test = train_test_split(dataK7_full.drop('labels', axis = 1), dataK7_full['labels'],
                                                    test_size = 0.2, random_state = 42)